In [6]:
import torch



In [7]:
def KL_diag_gaussian(mu_1,diag_1,mu_2,diag_2):
    ratio=diag_1/diag_2
    return torch.sum(0.5*(mu_1-mu_2)**2/diag_2)+0.5*torch.sum(ratio)-0.5*torch.sum(torch.log(ratio))-mu_1.size(0)/2



In [8]:
mu_1=torch.tensor([0.0,0.0])
diag_1=torch.tensor([1.0,1.0])
mu_2=torch.tensor([1.0,2.0])
diag_2=torch.tensor([0.5,0.5])

print(KL_diag_gaussian(mu_1,diag_1,mu_2,diag_2))

tensor(5.3069)


In [23]:
def low_rank_cov_inverse(L,sigma):
    # L is D*R
    dim=L.size(0)
    rank=L.size(1)
    var=sigma**2
    inverse_var=1.0/var
    inner_inverse=torch.inverse(torch.diag(torch.ones([rank]))+inverse_var*(L.t())@L)
    return inverse_var*torch.diag(torch.ones([dim]))-inverse_var**2*L@inner_inverse@L.t()

def low_rank_gaussian_logdet(L,sigma):
    dim=L.size(0)
    rank=L.size(1)
    var=sigma**2
    inverse_var=1.0/var
    return torch.logdet(torch.diag(torch.ones([rank]))+inverse_var*(L.t())@L)+dim*tf.log(var)
    
    

def low_rank_gaussian_sample(mu,L,sigma,amount):
    # L is D*R
    dim=L.size(0)
    rank=L.size(1)
    eps_z=torch.randn([amount,rank])
    eps=torch.randn([amount,dim])
    
    return eps_z@L.t()+eps*sigma+mu
    

def KL_low_rank_gaussian_with_diag_gaussian(mu_1,L_1,sigma_1,mu_2,sigma_2):
    dim_1=L_1.size(0)
    rank_1=L_1.size(1)
    var_1=sigma_1**2
    inverse_var_1=1.0/var_1
    logdet_1=torch.logdet(torch.diag(torch.ones([rank_1]))+inverse_var_1*(L_1.t())@L_1)+dim_1*torch.log(var_1)
    cov_1=L_1@L_1.t()+torch.diag(torch.ones([dim_1]))*var_1
    mu_diff=(mu_1-mu_2).view(-1,1)
    var_2=sigma_2**2
    return -0.5*(logdet_1-dim_1*torch.log(var_2)+dim_1-(1/var_2)*torch.trace(cov_1)-(1/var_2)*mu_diff.t()@mu_diff)
    


def KL_low_rank_gaussian_with_low_rank_gaussian(mu_1,L_1,sigma_1,mu_2,L_2,sigma_2):
    dim_1=L_1.size(0)
    rank_1=L_1.size(1)
    var_1=sigma_1**2
    inverse_var_1=1.0/var_1
    logdet_1=torch.logdet(torch.diag(torch.ones([rank_1]))+inverse_var_1*(L_1.t())@L_1)+dim_1*torch.log(var_1)
    cov_1=L_1@L_1.t()+torch.diag(torch.ones([dim_1]))*var_1
    
    
    dim_2=L_2.size(0)
    rank_2=L_2.size(1)
    var_2=sigma_2**2
    inverse_var_2=1.0/var_2
    logdet_2=torch.logdet(torch.diag(torch.ones([rank_2]))+inverse_var_2*(L_2.t())@L_2)+dim_1*torch.log(var_2)
    
    inner_inverse_2=torch.inverse(torch.diag(torch.ones([rank_2]))+inverse_var_2*(L_2.t())@L_2)
    cov_inverse_2=inverse_var_2*torch.diag(torch.ones([dim_2]))-inverse_var_2**2*L_2@inner_inverse_2@L_2.t()
    
    mu_diff=(mu_1-mu_2).view(-1,1)
    return -0.5*(logdet_1-logdet_2+dim_1-torch.trace(cov_1@cov_inverse_2)-mu_diff.t()@ cov_inverse_2@mu_diff)
    

def general_kl_divergence(mu_1,cov_1,mu_2,cov_2):
    mu_diff=(mu_1-mu_2).view(-1,1)
    cov_2_inverse=torch.inverse(cov_2)
    return -0.5*(torch.logdet(cov_1)-torch.logdet(cov_2)+mu_1.size(0)-torch.trace(cov_1@cov_2_inverse)-mu_diff.t()@cov_2_inverse@mu_diff)
   

In [11]:
L_1=torch.randn(4,2)
sigma_1=torch.tensor(1.0)
mu_1=torch.randn(4)
print(mu_1)
print(L_1.t())
cov_1=L_1@(L_1.t())+torch.diag(torch.ones(4))*sigma_1**2
print(cov_1)
L_2=torch.randn(4,2)
sigma_2=torch.tensor(1.0)
mu_2=torch.randn(4)
print(mu_2)
cov_2=L_2@L_2.t()+torch.diag(torch.ones(4))*sigma_2**2
print(cov_2)

tensor([-1.3405, -0.6657,  1.0196, -1.7288])
tensor([[-1.0631, -0.2356,  0.2253,  0.3571],
        [ 1.1482, -0.2559, -0.8499, -0.7693]])
tensor([[ 3.4485, -0.0433, -1.2154, -1.2629],
        [-0.0433,  1.1210,  0.1644,  0.1127],
        [-1.2154,  0.1644,  1.7732,  0.7343],
        [-1.2629,  0.1127,  0.7343,  1.7193]])
tensor([-0.2707, -1.3722,  0.9919,  0.4002])
tensor([[ 1.8812, -1.0184,  0.6714, -0.4969],
        [-1.0184,  2.1841, -0.8154,  0.6533],
        [ 0.6714, -0.8154,  1.7325, -0.8212],
        [-0.4969,  0.6533, -0.8212,  2.1670]])


In [12]:
print(general_kl_divergence(mu_1,cov_1,mu_2,cov_2))

tensor([[3.1407]])


In [19]:
print(KL_low_rank_gaussian_with_low_rank_gaussian(mu_1,L_1,sigma_1,mu_2,L_2,sigma_2))

tensor([[3.1407]])


In [20]:
L_1=torch.randn(4,2)
sigma_1=torch.tensor(1.0)
mu_1=torch.randn(4)
print(mu_1)
print(L_1.t())
cov_1=L_1@(L_1.t())+torch.diag(torch.ones(4))*sigma_1**2
print(cov_1)
sigma_2=torch.tensor(1.0)
mu_2=torch.randn(4)
print(mu_2)
cov_2=torch.diag(torch.ones(4))*sigma_2**2
print(cov_2)

tensor([ 0.1827,  0.2901, -1.3069, -0.9069])
tensor([[-1.1832, -0.2267, -0.4892, -0.0278],
        [ 0.3812, -0.8335, -1.0120, -1.4139]])
tensor([[ 2.5452, -0.0495,  0.1931, -0.5060],
        [-0.0495,  1.7461,  0.9544,  1.1848],
        [ 0.1931,  0.9544,  2.2636,  1.4446],
        [-0.5060,  1.1848,  1.4446,  3.0000]])
tensor([ 1.3834, -0.2485,  0.6343, -1.0809])
tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])


In [26]:
print(low_rank_gaussian_sample(mu_1,L_1,sigma_1,2).size())

torch.Size([2, 4])


In [21]:
print(general_kl_divergence(mu_1,cov_1,mu_2,cov_2))

tensor([[4.2596]])


In [24]:
print(KL_low_rank_gaussian_with_diag_gaussian(mu_1,L_1,sigma_1,mu_2,sigma_2))

tensor([[4.2596]])
